In [1]:
import sys
assert sys.version_info >= (3, 5)
import sklearn
assert sklearn.__version__ >= "0.20"
import tensorflow as tf
import string
import re
assert tf.__version__ >= "2.0"
import numpy as np
import os
import keras
import random
import subprocess
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
np.random.seed(42)
import torch

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
from transformers import MarianMTModel, MarianTokenizer

In [2]:
text_file = "C:\\Users\\Emanuel\\Desktop\\AD\\TP2\\por.txt"
with open(text_file, encoding="utf8") as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    en, pt = line.split("\t")
    text_pairs.append((f">>por<< {en}", pt))
del lines

text_pairs = random.sample(text_pairs, 100)
en_text = [pair[0] for pair in text_pairs]
pt_text = [pair[1] for pair in text_pairs]

In [3]:
model_name = "C:\\Users\\Emanuel\\Downloads\\opus-mt-tc-big-en-pt"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

In [4]:
bleu_pt_list = []
bleu_translation_list = []
for i in range(len(en_text)):
    # Dividir o pares em listas de palavras, meter tudo em lower case e retirar start/end
    expected_pt_text = pt_text[i].lower().split(" ")
    en_instance = en_text[i]

    if len(expected_pt_text) <= 4:
        continue
        
    translated = model.generate(**tokenizer([en_instance], return_tensors="pt", padding=True))
    translated = tokenizer.decode(translated[0], skip_special_tokens=True)
    translated = translated.lower().split(" ")
    # Filtrar strings vazias
    expected_pt_text = list(filter(None, expected_pt_text))
    translated = list(filter(None, translated))
    bleu_pt_list.append([expected_pt_text])
    bleu_translation_list.append(translated)

# Calcular 4-gram BLEU
gram4 = corpus_bleu(bleu_pt_list, bleu_translation_list, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=SmoothingFunction().method1)
print(f'4-gram BLEU: {round(np.mean(gram4),3)}')

4-gram BLEU: 0.46


In [5]:
for i in range(10):
    print(" ".join(bleu_pt_list[i][0]))
    print(" ".join(bleu_translation_list[i]))
    print("-")

somos as primeiras a chegar.
somos os primeiros a chegar.
-
tom tomou algumas decisões ruins recentemente.
tom tomou algumas decisões ruins recentemente.
-
o natal cai num domingo este ano.
o natal cai no domingo deste ano.
-
pensei que você precisava de dinheiro.
pensei que precisavas de dinheiro.
-
o vapor embaçou meus óculos.
o vapor enevoou os meus óculos.
-
essa é uma bela de uma suposição.
é uma suposição muito grande.
-
esse piano é muito barato.
este piano é muito barato.
-
tom derramou água fria sobre a cabeça.
tom derramou água fria sobre sua cabeça.
-
o tom estava um pouco embriagado.
tom era um pouco gordinho.
-
"tom é seu namorado, não é?"
"o tom é o teu namorado, não é?"
-
